In [4]:
import importlib

import common

importlib.reload(common)

<module 'common' from 'C:\\Users\\j\\IdeaProjects\\fiit_masters_thesis_project\\jupyter\\common.py'>

In [18]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy

from common import create_engine
from common import display_all
from common import figsize
from common import save_df


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters() # converters e.g. for datetime in plots

In [29]:
RANDOM_STATE = 123
np_random = np.random.RandomState(RANDOM_STATE)

### Load dataset

In [6]:
data = load_df('final_data.pickle')

df = pd.DataFrame()

### Prepare features group 1

### Prepare features group 2

### Split to 3 groups

In [38]:
def split_data(df, p1_size=2, p2_size=2, p3_size=1, shuffle=True):
    parts = p1_size + p2_size + p3_size
    
    if shuffle:
        splits = np.array_split(df.sample(frac=1, random_state=np_random), parts)
    else:
        splits = np.array_split(df, parts)
            
    df1 = pd.DataFrame(np.concatenate(splits[:p1_size]), columns=df.columns)
    df2 = pd.DataFrame(np.concatenate(splits[p1_size: p1_size + p2_size]), columns=df.columns)
    df3 = pd.DataFrame(np.concatenate(splits[p1_size + p2_size:]), columns=df.columns)
   
    return df1, df2, df3

df1, df2, df3 = split_data(df)

In [39]:
f'{len(df1)} + {len(df2)} + {len(df3)} == {len(df)}'

'61572 + 61572 + 30785 == 153929'

---

### Popularity prediction

### Fake new  detection

-----

### Method execution

---

### Results analysis